# ImmoEliza - XG Boost

After an exploration of several models, XG Boost turned out to provide the best results. Therefore this notebook is a seperate flow of this model.

If you want to rerun the model for a different property type (see below), you will need to import the dataframe again.

**Caveats:**
* At the moment this model only includes numerical variables (such as number of bedrooms or area of the plot) not categorical variables (such as building condition, heating type or EPC). 
* The model does not use any form of scaling.

In [13]:
#import libraries 
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xg

In [14]:
#import dataframe and display first rows and the total number of columns and rows
df = pd.read_csv('../data/saved_data.csv', index_col=0)
display(df.head())
df.shape

,property_type,property_subtype,immo code,Location,Postcode,Province,Region,Price,Address,Energy class,...,Construction year,Building condition,Bedrooms,Bathrooms,Shower rooms,Office,Toilets,Kitchen type,Heating type,Price_orig
0,house,house,10666428,essen,2910,Antwerpen,Flanders,469000,Heidestatiestraat 26,D,...,0,0,3,1,0,0,2,Installed,0,€469000
1,house,house,10666426,bierges,1301,Brabant wallon,Wallonia,1395000,"Rue de Wavre, 27",C,...,1988,Good,5,2,3,1,4,Hyper equipped,Fuel oil,€1395000
2,apartment,apartment,10666424,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,0,Good,2,2,0,0,2,Installed,Electric,€285000
3,house,house,10666423,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,0,Good,2,2,0,0,2,Installed,Electric,€285000
4,apartment,apartment,10666422,antwerp,2000,Antwerpen,Flanders,425000,"Grote steenweg, 511",C,...,0,As new,2,1,0,0,0,Installed,0,€425000


(6623, 27)

### Select properties by (sub)type
You can hardly compare a kot to a castle. Therefore you can run the model for different property (sub)types. This model is limited to housing types so commercial and industrial buildings are excluded. It uses the types and subtypes set by Immoweb. There are two main types, 'house' and 'apartment' with the following subtypes: 

* house: house, villa, mixed-use-building, exceptional-property, other-property, country-cottage, mansion, town-house, bungalow, farmhouse, new-real-estate-project-houses, chalet, manor-house, castle
* apartment: apartment, service-flat, duplex, triplex, apartment-block, loft, ground-floor, new-real-estate-project-apartments, kot, penthouse, flat-studio

By changing the codeblock below, you can select which (sub)types you want to include (==) or exclude (!=). By default castles are excluded from the model. If you want to change this selection, you should start by reloading the original data by rerunning the codeblock above.

In [15]:

#df = df[df.property_type == 'apartment']
df = df[df.property_type == 'house']
df = df[df.property_subtype != 'castle'] #by default we exclude castles from our model 

In [16]:
print(df.shape)
display(df.head())

(4196, 27)


,property_type,property_subtype,immo code,Location,Postcode,Province,Region,Price,Address,Energy class,...,Construction year,Building condition,Bedrooms,Bathrooms,Shower rooms,Office,Toilets,Kitchen type,Heating type,Price_orig
0,house,house,10666428,essen,2910,Antwerpen,Flanders,469000,Heidestatiestraat 26,D,...,0,0,3,1,0,0,2,Installed,0,€469000
1,house,house,10666426,bierges,1301,Brabant wallon,Wallonia,1395000,"Rue de Wavre, 27",C,...,1988,Good,5,2,3,1,4,Hyper equipped,Fuel oil,€1395000
3,house,house,10666423,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,0,Good,2,2,0,0,2,Installed,Electric,€285000
6,house,house,10666419,grace-hollogne,4460,Liï¿½ge,Wallonia,189000,Avenue Louis Libert 1,E,...,1934,Good,2,1,0,0,0,Semi equipped,Fuel oil,€189000
7,house,villa,10666418,aywaille,4920,Liï¿½ge,Wallonia,399999,Avenue Louis Libert 1,C,...,2004,To be done up,4,1,0,1,0,Semi equipped,Fuel oil,€399999


### Define model variables


In [17]:
#int_columns=['Bedrooms', 'Furnished', 'Terrace', 'Terrace surface', 'Surface of the plot', 'Living room surface', 'Number of frontages', 'Construction year', 'Bathrooms', 'Shower rooms', 'Office', 'Toilets']
int_columns=['Bedrooms', 'Terrace', 'Terrace surface', 'Surface of the plot', 'Living room surface', 'Number of frontages', 'Construction year', 'Bathrooms', 'Shower rooms', 'Office', 'Toilets', 'Primary energy consumption']
#int_columns=["Bedrooms", "Bathrooms", "Toilets"]

X = df[int_columns].to_numpy()
y = df.Price.to_numpy().reshape(-1,1)
print(type(X))
print(X)


<class 'numpy.ndarray'>
[[  3.   1.   0. ...   0.   2. 390.]
 [  5.   1.  60. ...   1.   4. 178.]
 [  2.   0.   0. ...   0.   2. 299.]
 ...
 [  6.   1.   0. ...   0.   3. 199.]
 [  4.   1.   0. ...   0.   0. 161.]
 [  2.   1.  60. ...   1.   4. 197.]]


### Split the dataset

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=41)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

Shape of X_train:  (3356, 12)
Shape of X_test:  (840, 12)
Shape of y_train:  (3356, 1)
Shape of y_test:  (840, 1)


##  Load and fit the model (with Sklearn)

In [19]:
XG_model = "XG Boost"

XG_regressor = xg.XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 20)
"""X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)"""
XG_regressor.fit(X_train, y_train)
XG_train_score = XG_regressor.score(X_train, y_train)
XG_test_score = XG_regressor.score(X_test,y_test)
XG_y_pred = XG_regressor.predict(X_test)
XG_R_MSE = mean_squared_error(y_test, XG_y_pred, squared = False)

[15:38:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
"""pseudocode

XG_regressor opslaan
in API model importeren 
XG_regressor.predict toepassen op API-input"""

'pseudocode\n\nXG_regressor opslaan\nin API model importeren \nXG_regressor.predict toepassen op API-input'

### Scores

In [21]:
model_score_list = ["model", "train_score", "test_score", "R_MSE"]

data = [[XG_model, XG_train_score, XG_test_score, XG_R_MSE]]
newscore_df = pd.DataFrame(data, columns=model_score_list)
#score_df = score_df.join(newscore_df, on=['model'], how='right')
#score_df ["train_score", "test_score", "R_MSE"] = newscore_df ["train_score", "test_score", "R_MSE"]
score_df=newscore_df
display(score_df)

,model,train_score,test_score,R_MSE
0,XG Boost,0.853098,0.624438,338979.294959


### Save Model

In [22]:
XG_regressor.save_model('../models/xg_model.json')

### Get a random sample

In [43]:
test_sample = (
        df[int_columns]
        .sample()
        .to_dict(orient="dict")
)
display(test_sample)
print(type(test_sample))
test_df = pd.DataFrame.from_dict(test_sample)
print(XG_regressor.predict(test_df))
display(test_df)

{'Bedrooms': {3409: 3},
 'Terrace': {3409: 1},
 'Terrace surface': {3409: 42},
 'Surface of the plot': {3409: 310},
 'Living room surface': {3409: 12},
 'Number of frontages': {3409: 3},
 'Construction year': {3409: 1965},
 'Bathrooms': {3409: 1},
 'Shower rooms': {3409: 0},
 'Office': {3409: 0},
 'Toilets': {3409: 1},
 'Primary energy consumption': {3409: 388.0}}

<class 'dict'>
[259416.11]


,Bedrooms,Terrace,Terrace surface,Surface of the plot,Living room surface,Number of frontages,Construction year,Bathrooms,Shower rooms,Office,Toilets,Primary energy consumption
3409,3,1,42,310,12,3,1965,1,0,0,1,388.0
